# Tests

In [ ]:
from nba_api.stats.endpoints import playbyplayv3
pbp = playbyplayv2.PlayByPlayV2(game_id='0022200001')  # Example Game ID
df = pbp.get_data_frames()[0]
print(df)

In [22]:
from nba_api.stats.endpoints import boxscoreadvancedv2 as bsav2
pbs = bsav2.BoxScoreAdvancedV2(game_id='0022200001')
df = pbs.get_data_frames()[0]
print(df)

       GAME_ID     TEAM_ID TEAM_ABBREVIATION     TEAM_CITY  PLAYER_ID  \
0   0022200001  1610612755               PHI  Philadelphia     202699   
1   0022200001  1610612755               PHI  Philadelphia     200782   
2   0022200001  1610612755               PHI  Philadelphia     203954   
3   0022200001  1610612755               PHI  Philadelphia    1630178   
4   0022200001  1610612755               PHI  Philadelphia     201935   
5   0022200001  1610612755               PHI  Philadelphia    1626149   
6   0022200001  1610612755               PHI  Philadelphia    1629001   
7   0022200001  1610612755               PHI  Philadelphia    1627863   
8   0022200001  1610612755               PHI  Philadelphia    1627777   
9   0022200001  1610612755               PHI  Philadelphia    1629680   
10  0022200001  1610612755               PHI  Philadelphia    1627788   
11  0022200001  1610612755               PHI  Philadelphia    1629003   
12  0022200001  1610612755               PHI  Phila

In [ ]:
from nba_api.stats.endpoints import boxscoreadvancedv2 as bs
from nba_api.stats.endpoints import playbyplayv2 as pbp
from collections import defaultdict
import pandas as pd

game_id = '0022200001'
bs_df = bsav2.BoxScoreAdvancedV2(game_id=game_id).get_data_frames()[0]
pbp_df = playbyplayv2.PlayByPlayV2(game_id=game_id).get_data_frames()[0]

players_df = bs_df[["PLAYER_NAME"]].copy()
players_df["ACTIVE"] = bs_df["START_POSITION"] != ""
players_df["IOC%"] = 0.0

print(players_df)

name_to_idx = {name: i for i, name in enumerate(players_df['PLAYER_NAME'])}

# filter to only subs
subs = pbp_df[pbp_df['EVENTMSGTYPE'] == 8]

for _, row in subs.iterrows():
    out_player = row['PLAYER1_NAME']
    in_player  = row['PLAYER2_NAME']
    
    # flip flags
    players_df.loc[players_df['PLAYER_NAME'] == out_player, 'ACTIVE'] = False
    players_df.loc[players_df['PLAYER_NAME'] == in_player,  'ACTIVE'] = True
    
    # print what happened
    print(f"Play {_}: 🔁 Substitution: {in_player} in, {out_player} out")
    print(players_df[['PLAYER_NAME','ACTIVE']])

print(players_df)

# Code

In [60]:
# Import Statements
from nba_api.stats.endpoints import boxscoreadvancedv2 as bs
from nba_api.stats.endpoints import playbyplayv2 as pbp
from collections import defaultdict
import pandas as pd
import re

# Function to reurn a list of game ids being used. 
def get_game_ids():
    game_ids = ['0022200001'] # Currently just one game being used for testing
    return game_ids

# Function to get the player list used to create custom statistics
def get_players_df(game_id):
    bs_df = bs.BoxScoreAdvancedV2(game_id=game_id).get_data_frames()[0] # Get players from the box score
    players_df = bs_df[["PLAYER_NAME", "TEAM_ABBREVIATION"]].copy()
    players_df["ACTIVE"] = bs_df["START_POSITION"] != "" # Initilize active players
    return players_df

# Give the play by play of the given game
def get_pbp_df(game_id):
    pbp_df = pbp.PlayByPlayV2(game_id=game_id).get_data_frames()[0]
    return pbp_df

# Calculate the stats of the game
def calc_stats(pbp, players, game_id):
    pbp[['HOMEDESCRIPTION','VISITORDESCRIPTION','NEUTRALDESCRIPTION']] = \
        pbp[['HOMEDESCRIPTION','VISITORDESCRIPTION','NEUTRALDESCRIPTION']].fillna('')
    players['LAST'] = players['PLAYER_NAME'].str.split().str[-1]
    last_to_full = dict(zip(players['LAST'], players_df['PLAYER_NAME']))
    # Set custom stats
    players["IOC"] = 0
    players["POS"] = 0
    #players["AIOC"] = 0.0

    players["POINTS"] = 0
    players["ASSISTS"] = 0
     # Counters for “excluded” possessions
    poss_count  = defaultdict(int)
    poss_points = defaultdict(int)

    # Possession‐level state
    current_team    = None    # 'home' or 'away'
    current_points  = 0
    current_scorers = set()
    current_assists = set()
    
    # Loop though plays of the game for analysis
    for _, row in pbp.iterrows(): 
        # 1) SUBSTITUTIONS: flip ACTIVE flags
        if row['EVENTMSGTYPE'] == 8:
            out_player = row['PLAYER1_NAME']
            in_player  = row['PLAYER2_NAME']
            
            # flip flags
            players.loc[players['PLAYER_NAME'] == out_player, 'ACTIVE'] = False
            players.loc[players['PLAYER_NAME'] == in_player,  'ACTIVE'] = True
            #print("Substitution")
            #print(players)
            continue
            
        # Merge descriptions
        desc = row['HOMEDESCRIPTION'] + row['VISITORDESCRIPTION'] + row['NEUTRALDESCRIPTION']
        scorer = None
        assister = None
        # ——— Field Goals Made ———
        if row['EVENTMSGTYPE'] == 1:
            #print(desc)

            # 4a) Extract scorer’s last name (first token)
            last = desc.split()[0]
            scorer = last_to_full.get(last)
            idx = players_df.index[players_df['PLAYER_NAME'] == scorer][0]
            play_pts = players.at[idx, "POINTS"]
            if scorer:
                #print(scorer)
                # 4b) Extract points from "(n PTS)"
                m_pts = re.search(r'\((\d+)\s+PTS\)', desc)
                if m_pts:
                    pts = int(m_pts.group(1))
                    #players.loc[players['PLAYER_NAME'] == scorer, "POINTS"] = pts
                    players.at[idx, "POINTS"] = pts
                    play_pts = pts - play_pts
                #print(players.loc[scorer])
    
            # 4c) Extract assist if present: "(Lastname n AST)"
            assister = None
            m_ast = re.search(r'\((\w+)\s+(\d+)\s+AST\)', desc)
            if m_ast:
                last_a  = m_ast.group(1)
                count_a = int(m_ast.group(2))
                assister = last_to_full.get(last_a)
                if assister:
                    idx = players_df.index[players_df['PLAYER_NAME'] == assister][0]
                    players.at[idx, 'ASSISTS'] += 1

            # Add IOC to players who were on the team that scored and active but weren't the scorer or assister
            if scorer:
                team = players.loc[players.PLAYER_NAME == scorer, 'TEAM_ABBREVIATION'].iloc[0]
                mask = (
                    (players['TEAM_ABBREVIATION'] == team) &
                    (players['ACTIVE']) &
                    (players['PLAYER_NAME'] != scorer)
                )
                if assister:
                    mask &= (players['PLAYER_NAME'] != assister)
                players.loc[mask, 'IOC'] += play_pts

        # ——— Free Throws Made ———
        if row['EVENTMSGTYPE'] == 3 and 'PTS' in desc:
            #print("FT:", desc)
        
            # 1) Extract shooter’s last name & full name
            last = desc.split()[0]
            shooter = last_to_full.get(last)
            if not shooter:
                continue
        
            # 2) Compute how many new points this FT added
            idx = players_df.index[players_df['PLAYER_NAME'] == shooter][0]
            before_pts = players.at[idx, "POINTS"]
            m_pts = re.search(r'\((\d+)\s+PTS\)', desc)
            if not m_pts:
                continue
            total_pts = int(m_pts.group(1))
            players.at[idx, "POINTS"] = total_pts
            play_pts = total_pts - before_pts
        
            # 3) No assist parsing (FTs don’t carry assists)
        
            # 4) IOC for everyone else on that team who’s active
            team = players.loc[players.PLAYER_NAME == shooter, 'TEAM_ABBREVIATION'].iloc[0]
            mask = (
                (players['TEAM_ABBREVIATION'] == team) &
                (players['ACTIVE']) &
                (players['PLAYER_NAME'] != shooter)
            )
            # add only the incremental FT points
            players.loc[mask, 'IOC'] += play_pts
            #print(f"  {shooter} hit a free throw for {play_pts} pt → IOC +{play_pts} to:",
                  #players.loc[mask, 'PLAYER_NAME'].tolist())


        is_def_reb = (row['EVENTMSGTYPE'] == 4 and 'Defensive' in desc)
        if row['EVENTMSGTYPE'] in (1, 5, 6) or is_def_reb:
            scorer = None
            assister = None
            # 1) figure out which team just had the ball
            offense_abbr = None
    
            if row['EVENTMSGTYPE'] == 1:   # made FG
                last = desc.split()[0]
                scorer = last_to_full.get(last)
                if scorer:
                    offense_abbr = players.loc[
                        players.PLAYER_NAME == scorer, 'TEAM_ABBREVIATION'
                    ].iloc[0]
    
            elif row['EVENTMSGTYPE'] == 5: # turnover
                last = desc.split()[0]
                turner = last_to_full.get(last)
                if turner:
                    offense_abbr = players.loc[
                        players.PLAYER_NAME == turner, 'TEAM_ABBREVIATION'
                    ].iloc[0]
    
            elif is_def_reb:             # defensive rebound
                last = desc.split()[0]
                rebounder = last_to_full.get(last)
                if rebounder:
                    # possession ended for the *other* team, so rebounder’s team is new offense
                    offense_abbr = players.loc[
                        players.PLAYER_NAME == rebounder, 'TEAM_ABBREVIATION'
                    ].iloc[0]
    
            # 2) now credit POS and IOC
            if offense_abbr:
                # mask of teammates on court
                base_mask = (
                    (players['TEAM_ABBREVIATION'] == offense_abbr) &
                    (players['ACTIVE'])
                )
                
                # Build exclusion list
                excl = {scorer}
                if assister:
                    excl.add(assister)
                
                # Final mask: exclude scorer and assister
                mask = base_mask & (~players['PLAYER_NAME'].isin(excl))
                
                # Now increment POS and IOC only for those players
                players.loc[mask, 'POS'] += 1
    
            # 3) reset for next possession
            current_points = 0
            current_scorers.clear()
            current_assists.clear()

            #print(f"Event: {row['EVENTMSGTYPE']}")
            #print(f"Possession Ended!")
            #print(f"    Scorer:   {scorer}")
            #print(f"    Assister: {assister}")


    # Remove the helper column
    players.drop(columns='LAST', inplace=True)
    return players

def main():
    game_ids = get_game_ids()
    for game_id in game_ids: # Loop though game ids to make an analysis for each game
        players = get_players_df(game_id)
        pbp = get_pbp_df(game_id)
        calc_players = calc_stats(pbp = pbp, players = players, game_id = game_id)
        print(calc_players)
        

if __name__ == "__main__":
    main()

          PLAYER_NAME TEAM_ABBREVIATION  ACTIVE  IOC  POS  POINTS  ASSISTS
0       Tobias Harris               PHI    True   66   28      18        0
1         P.J. Tucker               PHI    True   71   29       6        0
2         Joel Embiid               PHI   False   11    7      26        5
3        Tyrese Maxey               PHI    True   66   30      21        2
4        James Harden               PHI    True   50   30      35        7
5    Montrezl Harrell               PHI    True   99   37       2        0
6   De'Anthony Melton               PHI   False   45   15       5        0
7    Danuel House Jr.               PHI   False   39   13       0        0
8       Georges Niang               PHI   False    8    3       3        1
9    Matisse Thybulle               PHI    True   30   11       0        0
10     Furkan Korkmaz               PHI   False    0    0       0        0
11       Shake Milton               PHI   False    0    0       0        0
12          Paul Reed    